In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

import os
if not os.path.isdir('saved_models_{}'.format('gan')):
    os.mkdir('saved_models_{}'.format('gan'))
f = open('saved_models_{}/log_collapse1.txt'.format('gan'), mode='w')
import torch.utils.data as Data
import cv2


from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam #optimizer of keras

import matplotlib.pyplot as plt
from PIL import Image
import sys

import numpy as np
import os
class GAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels) #shape of image
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5) #optimizer of gan

        # Build and compile the discriminator,only to keras
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))  #Input():用来实例化一个keras张量
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5

        # X_train = np.expand_dims(X_train, axis=3)  #expand_dims用于扩充数组形状
        print(np.shape(X_train))

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Train the generator (to have the discriminator label samples as valid)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[1], g_loss))
                sample_num=5000

                if global_step % sample_interval == 0:
                    self.mode_drop(X_test,y_test,sample_num, global_step)


    def mode_drop(self, x_test,y_test,sample_num, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('step:' + str(global_step))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 1,704,961
Trainable params: 1,704,961
Non-trainable params: 0
_________________________________________________________________


/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.722421, acc.: 54.69%] [G loss: 0.634639]
epoch:0 step:2 [D loss: 0.625889, acc.: 57.03%] [G loss: 0.607043]
epoch:0 step:3 [D loss: 0.565688, acc.: 58.59%] [G loss: 0.656417]
epoch:0 step:4 [D loss: 0.539331, acc.: 64.06%] [G loss: 0.724627]
epoch:0 step:5 [D loss: 0.489601, acc.: 69.53%] [G loss: 0.742720]
epoch:0 step:6 [D loss: 0.496257, acc.: 71.88%] [G loss: 0.784894]
epoch:0 step:7 [D loss: 0.521062, acc.: 71.88%] [G loss: 0.888948]
epoch:0 step:8 [D loss: 0.478658, acc.: 71.09%] [G loss: 0.898402]
epoch:0 step:9 [D loss: 0.477769, acc.: 73.44%] [G loss: 0.919063]
epoch:0 step:10 [D loss: 0.479139, acc.: 71.09%] [G loss: 0.949653]
epoch:0 step:11 [D loss: 0.421680, acc.: 83.59%] [G loss: 1.013812]
epoch:0 step:12 [D loss: 0.447405, acc.: 78.12%] [G loss: 0.990016]
epoch:0 step:13 [D loss: 0.442277, acc.: 81.25%] [G loss: 1.084875]
epoch:0 step:14 [D loss: 0.454726, acc.: 79.69%] [G loss: 1.098620]
epoch:0 step:15 [D loss: 0.415784, acc.: 82.81%] [G loss:

epoch:0 step:125 [D loss: 0.721289, acc.: 61.72%] [G loss: 1.489766]
epoch:0 step:126 [D loss: 0.749689, acc.: 59.38%] [G loss: 1.456690]
epoch:0 step:127 [D loss: 0.741318, acc.: 54.69%] [G loss: 1.425542]
epoch:0 step:128 [D loss: 0.701079, acc.: 54.69%] [G loss: 1.584983]
epoch:0 step:129 [D loss: 0.796261, acc.: 50.78%] [G loss: 1.404122]
epoch:0 step:130 [D loss: 0.718984, acc.: 56.25%] [G loss: 1.542346]
epoch:0 step:131 [D loss: 0.760926, acc.: 52.34%] [G loss: 1.425337]
epoch:0 step:132 [D loss: 0.625272, acc.: 60.16%] [G loss: 1.548638]
epoch:0 step:133 [D loss: 0.640235, acc.: 62.50%] [G loss: 1.519421]
epoch:0 step:134 [D loss: 0.602225, acc.: 67.19%] [G loss: 1.392839]
epoch:0 step:135 [D loss: 0.753010, acc.: 54.69%] [G loss: 1.436329]
epoch:0 step:136 [D loss: 0.615150, acc.: 63.28%] [G loss: 1.538992]
epoch:0 step:137 [D loss: 0.770045, acc.: 42.97%] [G loss: 1.461833]
epoch:0 step:138 [D loss: 0.681338, acc.: 61.72%] [G loss: 1.448668]
epoch:0 step:139 [D loss: 0.680076

...

epoch:49 step:38480 [D loss: 0.399281, acc.: 84.38%] [G loss: 3.214844]
epoch:49 step:38481 [D loss: 0.266406, acc.: 89.06%] [G loss: 2.740890]
epoch:49 step:38482 [D loss: 0.396214, acc.: 82.81%] [G loss: 2.595872]
epoch:49 step:38483 [D loss: 0.344082, acc.: 85.16%] [G loss: 2.707899]
epoch:49 step:38484 [D loss: 0.404009, acc.: 83.59%] [G loss: 2.258902]
epoch:49 step:38485 [D loss: 0.325880, acc.: 80.47%] [G loss: 3.530309]
epoch:49 step:38486 [D loss: 0.255660, acc.: 86.72%] [G loss: 5.348198]
epoch:49 step:38487 [D loss: 0.290592, acc.: 87.50%] [G loss: 4.632569]
epoch:49 step:38488 [D loss: 0.248187, acc.: 90.62%] [G loss: 2.994806]
epoch:49 step:38489 [D loss: 0.354527, acc.: 86.72%] [G loss: 5.027160]
epoch:49 step:38490 [D loss: 0.265020, acc.: 89.06%] [G loss: 4.471312]
epoch:49 step:38491 [D loss: 0.291370, acc.: 87.50%] [G loss: 2.847691]
epoch:49 step:38492 [D loss: 0.328054, acc.: 85.16%] [G loss: 2.800883]
epoch:49 step:38493 [D loss: 0.249181, acc.: 89.84%] [G loss: 3.

epoch:49 step:38595 [D loss: 0.279770, acc.: 88.28%] [G loss: 2.299343]
epoch:49 step:38596 [D loss: 0.348635, acc.: 84.38%] [G loss: 3.525058]
epoch:49 step:38597 [D loss: 0.360846, acc.: 85.16%] [G loss: 3.155296]
epoch:49 step:38598 [D loss: 0.310789, acc.: 85.16%] [G loss: 3.170573]
epoch:49 step:38599 [D loss: 0.244694, acc.: 89.84%] [G loss: 2.862066]
epoch:49 step:38600 [D loss: 0.301055, acc.: 86.72%] [G loss: 2.359544]
##############
[0.85701823 0.86364706 0.8164577  0.81104416 0.7763444  0.83127803
 0.87772699 0.80899449 0.8195672  0.81530491]
##########
epoch:49 step:38601 [D loss: 0.300656, acc.: 89.06%] [G loss: 3.335723]
epoch:49 step:38602 [D loss: 0.396423, acc.: 82.03%] [G loss: 3.713251]
epoch:49 step:38603 [D loss: 0.449398, acc.: 78.91%] [G loss: 3.169914]
epoch:49 step:38604 [D loss: 0.299663, acc.: 86.72%] [G loss: 3.328427]
epoch:49 step:38605 [D loss: 0.382566, acc.: 82.03%] [G loss: 3.255327]
epoch:49 step:38606 [D loss: 0.343112, acc.: 83.59%] [G loss: 3.53051

epoch:49 step:38709 [D loss: 0.297910, acc.: 89.06%] [G loss: 3.329345]
epoch:49 step:38710 [D loss: 0.357277, acc.: 82.03%] [G loss: 3.035904]
epoch:49 step:38711 [D loss: 0.251900, acc.: 89.06%] [G loss: 3.158366]
epoch:49 step:38712 [D loss: 0.330853, acc.: 86.72%] [G loss: 2.942727]
epoch:49 step:38713 [D loss: 0.253985, acc.: 87.50%] [G loss: 2.958361]
epoch:49 step:38714 [D loss: 0.292125, acc.: 86.72%] [G loss: 3.621078]
epoch:49 step:38715 [D loss: 0.309042, acc.: 83.59%] [G loss: 2.551534]
epoch:49 step:38716 [D loss: 0.317550, acc.: 85.16%] [G loss: 2.494732]
epoch:49 step:38717 [D loss: 0.388467, acc.: 81.25%] [G loss: 3.135513]
epoch:49 step:38718 [D loss: 0.234481, acc.: 90.62%] [G loss: 3.056872]
epoch:49 step:38719 [D loss: 0.264555, acc.: 86.72%] [G loss: 3.268542]
epoch:49 step:38720 [D loss: 0.231622, acc.: 89.06%] [G loss: 3.627468]
epoch:49 step:38721 [D loss: 0.238967, acc.: 88.28%] [G loss: 3.022273]
epoch:49 step:38722 [D loss: 0.292518, acc.: 85.16%] [G loss: 3.

epoch:49 step:38821 [D loss: 0.328754, acc.: 87.50%] [G loss: 5.289224]
epoch:49 step:38822 [D loss: 0.196828, acc.: 93.75%] [G loss: 4.058919]
epoch:49 step:38823 [D loss: 0.240532, acc.: 91.41%] [G loss: 4.009463]
epoch:49 step:38824 [D loss: 0.271904, acc.: 90.62%] [G loss: 4.550912]
epoch:49 step:38825 [D loss: 0.279085, acc.: 85.94%] [G loss: 4.685321]
epoch:49 step:38826 [D loss: 0.235435, acc.: 89.84%] [G loss: 4.906527]
epoch:49 step:38827 [D loss: 0.215753, acc.: 92.97%] [G loss: 3.955300]
epoch:49 step:38828 [D loss: 0.275441, acc.: 89.84%] [G loss: 3.748909]
epoch:49 step:38829 [D loss: 0.304294, acc.: 88.28%] [G loss: 4.099949]
epoch:49 step:38830 [D loss: 0.217989, acc.: 92.19%] [G loss: 3.219756]
epoch:49 step:38831 [D loss: 0.359735, acc.: 82.81%] [G loss: 3.257181]
epoch:49 step:38832 [D loss: 0.265051, acc.: 86.72%] [G loss: 3.370231]
epoch:49 step:38833 [D loss: 0.290591, acc.: 90.62%] [G loss: 3.390096]
epoch:49 step:38834 [D loss: 0.255071, acc.: 91.41%] [G loss: 2.

epoch:49 step:38939 [D loss: 0.239970, acc.: 92.19%] [G loss: 3.161451]
epoch:49 step:38940 [D loss: 0.386057, acc.: 78.91%] [G loss: 3.214644]
epoch:49 step:38941 [D loss: 0.296152, acc.: 81.25%] [G loss: 2.567756]
epoch:49 step:38942 [D loss: 0.306257, acc.: 86.72%] [G loss: 3.440923]
epoch:49 step:38943 [D loss: 0.360476, acc.: 84.38%] [G loss: 3.104663]
epoch:49 step:38944 [D loss: 0.357382, acc.: 83.59%] [G loss: 3.026015]
epoch:49 step:38945 [D loss: 0.326207, acc.: 85.16%] [G loss: 4.329074]
epoch:49 step:38946 [D loss: 0.245013, acc.: 90.62%] [G loss: 3.404981]
epoch:49 step:38947 [D loss: 0.311633, acc.: 87.50%] [G loss: 3.884705]
epoch:49 step:38948 [D loss: 0.238808, acc.: 89.06%] [G loss: 3.958358]
epoch:49 step:38949 [D loss: 0.288556, acc.: 87.50%] [G loss: 3.476619]
epoch:49 step:38950 [D loss: 0.269903, acc.: 88.28%] [G loss: 2.731886]
epoch:49 step:38951 [D loss: 0.287700, acc.: 85.16%] [G loss: 2.944839]
epoch:49 step:38952 [D loss: 0.301604, acc.: 89.06%] [G loss: 2.